In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from scipy.stats import binomtest

import uproot   # use of root files
import awkward as ak    # nested, variable sized data
import vector   # lorentz vectors
vector.register_awkward()
np.random.seed(42)

ModuleNotFoundError: No module named 'shap'

In [17]:
filename = "/cvmfs/atlas-nightlies.cern.ch/repo/data/data-art/ASG/DAOD_PHYSLITE/p6479/data18_13TeV.00348885.physics_Main.deriv.DAOD_PHYSLITE.r13286_p4910_p6479/DAOD_PHYSLITE.41578717._000256.pool.root.1"
compressed = "/eos/user/y/yolanney/compressedDAOD_PHYSLITE.pool.root"
# Load original
tree_orig = uproot.open({filename: "CollectionTree"})
el_pt_orig = tree_orig["AnalysisElectronsAuxDyn.pt"].array()
original = ak.flatten(el_pt_orig).to_numpy()

# Load compressed
tree_comp = uproot.open({compressed: "CollectionTree"})
el_pt_comp = tree_comp["AnalysisElectronsAuxDyn.pt"].array()
compressed = ak.flatten(el_pt_comp).to_numpy()

print(f'Original: {len(original):,} electrons')
print(f'Compressed: {len(compressed):,} electrons')

Original: 10,019 electrons
Compressed: 10,019 electrons


In [21]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import binomtest

def c2st_advanced(orig, comp, classifier_type='lr', n_trials=10, test_size=0.5, random_state=42):
    """
    Perform C2ST with a chosen advanced classifier.
    
    Args:
        orig (np.ndarray): Original samples (shape: [n_samples,]).
        comp (np.ndarray): Comparison samples (shape: [m_samples,]).
        classifier_type (str): 'lr', 'rf', 'xgb', 'svm', or 'mlp'.
        n_trials (int): Number of trials to average results.
        test_size (float): Fraction of data for testing.
        random_state (int): Random seed.
        
    Returns:
        dict: Mean accuracy, p-value, and H0 rejection status.
    """
    # Reshape to (n_samples, 1)
    orig = orig.reshape(-1, 1)
    comp = comp.reshape(-1, 1)
    
    # Balance datasets
    min_samples = min(len(orig), len(comp))
    X = np.vstack([orig[:min_samples], comp[:min_samples]])
    y = np.hstack([np.ones(min_samples), np.zeros(min_samples)])
    
    # Initialize classifier
    if classifier_type == 'lr':
        clf = LogisticRegression(max_iter=1000, random_state=random_state)
    elif classifier_type == 'rf':
        clf = RandomForestClassifier(n_estimators=100, random_state=random_state)
    elif classifier_type == 'xgb':
        clf = XGBClassifier(random_state=random_state)
    elif classifier_type == 'svm':
        clf = SVC(kernel='rbf', random_state=random_state)
    elif classifier_type == 'mlp':
        clf = MLPClassifier(hidden_layer_sizes=(64, 64, 64, 32), max_iter=500, random_state=random_state)
    else:
        raise ValueError("Classifier not supported. Use 'lr', 'rf', 'xgb', 'svm', or 'mlp'.")
    
    n_correct = 0
    n_total = 0
    for i in range(n_trials):
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state + i
        )
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        n_correct += np.sum(y_pred == y_test)
        n_total += len(y_test)

    if classifier_type == 'rf':
        importances = clf.feature_importances_
        print("RF Feature Importances:", importances)
    mean_accuracy = n_correct / n_total
    p_value = binomtest(n_correct, n_total, p=0.5, alternative='greater').pvalue
    
    return {
        "classifier": classifier_type,
        "mean_accuracy": mean_accuracy,
        "p_value": p_value,
        "reject_H0": p_value < 0.05
    }

# classifiers = ['lr', 'rf', 'xgb', 'svm', 'mlp']
classifiers = ['rf', 'mlp']

results = {}

# Print results
for clf, res in results.items():
    print(f"\nClassifier: {res['classifier'].upper()}")
    print(f"Mean Accuracy: {res['mean_accuracy']:.3f}")
    print(f"p-value: {res['p_value']:.5f}")
    print(f"Reject H0 (P≠Q)? {res['reject_H0']}")

RF Feature Importances: [1.]

Classifier: RF
Mean Accuracy: 0.915
p-value: 0.00000
Reject H0 (P≠Q)? True

Classifier: MLP
Mean Accuracy: 0.501
p-value: 0.35818
Reject H0 (P≠Q)? False


### Results (in case re-ran without completing)
took 14m 24.8s

Classifier: LR \
Mean Accuracy: 0.498 \
p-value: 0.83395 \
Reject H0 (P≠Q)? False

Classifier: RF \
Mean Accuracy: 0.915 \
p-value: 0.00000 \
Reject H0 (P≠Q)? True

Classifier: XGB \
Mean Accuracy: 0.509 \
p-value: 0.00000 \
Reject H0 (P≠Q)? True

Classifier: SVM \
Mean Accuracy: 0.495 \
p-value: 0.99909 \
Reject H0 (P≠Q)? False

Classifier: MLP, hidden_layer_sizes=(64, 32)\
Mean Accuracy: 0.500 \
p-value: 0.49622 \
Reject H0 (P≠Q)? False

### **Key Observations**
1. **Random Forest (RF) is the most powerful detector**  
   - Accuracy = 91.5% (far above chance) and `p-value ≈ 0` strongly suggests `original ≠ compressed`.  
   - RF’s decision trees likely captured nonlinear or high-order interactions missed by other models.

2. **Logistic Regression (LR), SVM, and MLP failed to detect differences**  
   - Accuracies ≈50% (`p-values > 0.05`) imply these classifiers couldn’t distinguish the distributions.  
   - **Possible reasons**:  
     - Differences are nonlinear (LR is linear; SVM’s RBF kernel may be misconfigured).  

3. **XGBoost’s paradoxical result**  
   - Accuracy = 50.9% (barely above chance) but `p-value = 0.000` suggests statistical significance 
   - This could indicate:  
     - Very subtle differences (XGBoost detected minimal but consistent patterns).  
     - Overfitting (check with more trials or cross-validation).
